<center>
    <h1>Verbal Explanation of Spatial Temporal GNNs for Traffic Forecasting</h1>
    <h2>Visualizing an Explained Instance of the Metr-LA Dataset</h2>
</center>

---

In this notebook an explained instance of Metr-LA is visualized.

In [1]:
import sys
import os

# Set the main path in the root folder of the project.
sys.path.append(os.path.join('..'))

In [2]:
# Settings for autoreloading.
%load_ext autoreload
%autoreload 2

In [3]:
from src.utils.seed import set_random_seed

# Set the random seed for deterministic operations.
SEED = 42
set_random_seed(SEED)

# 1 Loading the Data
The data is loaded.

In [4]:
import os

BASE_DATA_DIR = os.path.join('..', 'data', 'metr-la')

In [5]:
from src.data.data_extraction import get_adjacency_matrix

# Get the adjacency matrix
adj_matrix_structure = get_adjacency_matrix(
    os.path.join(BASE_DATA_DIR, 'raw', 'adj_mx_metr_la.pkl'))

# Get the header of the adjacency matrix, the node indices and the
# matrix itself.
header, node_ids_dict, adj_matrix = adj_matrix_structure

In [6]:
from src.data.data_extraction import get_locations_dataframe

# Get the dataframe containing the latitude and longitude of each sensor.
locations_df = get_locations_dataframe(
    os.path.join(BASE_DATA_DIR, 'raw', 'graph_sensor_locations_metr_la.csv'),
    has_header=True)

In [7]:
# Get the node positions dictionary.
node_pos_dict = { i: id for id, i in node_ids_dict.items() }

In [8]:
import os
import numpy as np

# Get the explained data.
x_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test.npy'))
y_test = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_test.npy'))

test_clusters = np.load(os.path.join(BASE_DATA_DIR, 'clustered', 'x_test.npy'))
test_translations = np.load(os.path.join(BASE_DATA_DIR, 'translated', 'test.npy'))

# Get the time information of the explained data.
x_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'x_test_time.npy'))
y_test_time = np.load(os.path.join(BASE_DATA_DIR, 'explained', 'y_test_time.npy'))

The instance speeds are turned into km/h.

In [9]:
# Turn the results in kilometers per hour.
from src.utils.config import MPH_TO_KMH_FACTOR


x_test[..., 0] = x_test[..., 0] * MPH_TO_KMH_FACTOR
y_test = y_test * MPH_TO_KMH_FACTOR

# 2 Visualization
The first instance explanation is visualized.

In [10]:
i = 0

sample_x = x_test[i]
clusters_x = test_clusters[i]
sample_y = y_test[i]
sample_x_time = x_test_time[i]
sample_y_time = y_test_time[i]

clusters_x = clusters_x.astype(object)

verbal_translation = test_translations[i]

In [11]:
_, n_timesteps, n_nodes, _ = y_test.shape

In [12]:
for c in np.unique(clusters_x):
    if c == -1:
        clusters_x[clusters_x == c] = ' '
    else:
        clusters_x[clusters_x == c] = f'cluster {c}'

In [13]:
clusters_y = (sample_y > 0).astype(np.int64).astype(object)
clusters_y[clusters_y == 0] = ' '
clusters_y[clusters_y == 1] = 'target'

In [14]:
from src.explanation.clustering.analyisis import get_node_values_with_clusters_and_location_dataframe

df = get_node_values_with_clusters_and_location_dataframe(sample_x[..., 0:1], clusters_x, node_pos_dict, locations_df, sample_x_time)

In [15]:
icons = {
    ' ': '',
    'cluster 0': 'star',
    'cluster 1': 'circle',
    'cluster 2':'heart',
    'cluster 3': 'play',
    'cluster 4': 'pause',
    'target': 'certified'}

In [16]:
df['icon'] = df['cluster'].apply(lambda x: icons[x])

In [17]:
print(verbal_translation)

San diego freeway at kms 11, 12, 13 and 14 was forecasted to see severe congestion on Wednesday, 04/06/2012, with an average speed of 24.04 km/h from 07:45 to 08:40. This occurred because of a series of congestions and a free flow.

To start, a contributing free flow materialized, at 107.75 km/h, on San Diego Freeway at kms 8 and 10 from 06:45 to 06:50.

Following this, a contributing congestion occurred, averaging at a speed of 82.83 km/h, on, another time, San Diego Freeway at kms 9 and 10 from 06:45 to 07:10.

Afterwards, a new contributing congestion took place, with an average speed of 88.65 km/h, on Ventura Freeway at kms 27, 28, 29, 30 and 31 occurring from 06:45 to 07:40. The congestion also took place on, once again, San Diego Freeway at kms 11, 12 and 13.

After this, a contributing severe congestion happened on, yet again, San Diego Freeway at kms 10, 11, 12, 13, 14 and 15 from 06:45 to 07:40 with an average speed of 19.70 km/h.

To conclude, yet a further contributing conge

In [30]:
from src.data.data_analysis import show_kepler_map

show_kepler_map(
    df, config_file_path='../config/kepler/metr-la/visualization_test_x_clusters.json')

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data'], 'id': '6w1zy4', 'na…

In [31]:
from src.explanation.clustering.analyisis import (
    get_node_values_with_clusters_and_location_dataframe)

df = get_node_values_with_clusters_and_location_dataframe(sample_y, clusters_y, node_pos_dict, locations_df, sample_y_time)

In [32]:
df['icon'] = df['cluster'].apply(lambda x: icons[x])

In [40]:
from src.data.data_analysis import show_kepler_map

show_kepler_map(
    df, config_file_path='../config/kepler/metr-la/visualization_test_y_clusters.json')

KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [{'dataId': ['data'], 'id': 'lbbs8b4ts', …